In [1]:
import pandas as pd
import re
import numpy as np
import altair as alt
import string 
import warnings
warnings.simplefilter('ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import recall_score, roc_auc_score, accuracy_score, plot_confusion_matrix, classification_report
#from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from regressors import stats
from sklearn.feature_selection import chi2
from itertools import compress
from joblib import dump, load
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [11]:
df = pd.read_csv("../../../Glentel Inc/HR Analytics - Documents/Capstone Data/ubc_mds_team_share/06_clean_data/0528_training_dataset.csv")

In [4]:
extract = pd.read_excel("../../../Glentel Inc/HR Analytics - Documents/Capstone Data/ubc_mds_team_share/01_resume_scan_data/manual_extraction_template.xlsx")

In [5]:
names_code = extract[['employee_code', 'employee_name']]

In [6]:
clf = load('../../model/finish_model/01_06_20_model.joblib')

In [13]:
def feature_drilldown(col):
    col = str(col)
    plot = alt.Chart(df).mark_bar().encode(
        alt.X("count()"),
        alt.Y(col+":N"))
    return plot

![](../../img/01_test_weights_interpertation.png)

In [20]:
feature_drilldown('hp_class')

alt.Chart(...)

In [21]:
feature_drilldown("blue collar_concentration")

alt.Chart(...)

In [22]:
feature_drilldown('no work')

alt.Chart(...)

In [23]:
feature_drilldown('cashier')

alt.Chart(...)

In [24]:
feature_drilldown('competitor_experience')

alt.Chart(...)

In [25]:
feature_drilldown('highest_degree')

alt.Chart(...)

In [26]:
feature_drilldown('background_highest_degree')

alt.Chart(...)

In [27]:
X = df.drop('hp_class', axis=1)

In [28]:
df_pred = df[['employee_code', 'hp_class'] ]

df_pred["pred"] = clf.predict(X).flatten()
#df_pred["pred_prob"] = clf.predict_proba(X).flatten()

pred_prob = clf.predict_proba(X)
prob_hp_1 = []
prob_hp_0 = []

for i in range(0, X.shape[0]):
    prob_hp_0.append(pred_prob[i][0])
    prob_hp_1.append(pred_prob[i][1])
    
df_pred["prob_hp_0"] = prob_hp_0
df_pred["prob_hp_1"] = prob_hp_1

df_pred["prob_hp_0"] = df_pred["prob_hp_0"].round(5)
df_pred["prob_hp_1"] = df_pred["prob_hp_1"].round(5)

In [29]:
df_pred = names_code.merge(df_pred, on="employee_code")

In [36]:
df_pred

,employee_code,employee_name,hp_class,pred,prob_hp_0,prob_hp_1
0,BDP,"Abdulali, Kaizar",0.0,0.0,0.60664,0.39336
1,F6Q,"Avila-Gil, Mario",1.0,1.0,0.05007,0.94993
2,MRN,"Amini, Kamyar",0.0,1.0,0.29376,0.70624
3,MXO,"Awan, Shoaib",0.0,0.0,0.87399,0.12601
4,N0Y,"Al-Khulaidy, Maha",0.0,0.0,0.84791,0.15209
5,N2R,"Arora, Ashish",1.0,0.0,0.90620,0.09380
6,N3O,"Abdi, Melika",0.0,0.0,0.91879,0.08121
7,N4B,"Atamaniuk, Jerry",0.0,1.0,0.48858,0.51142
8,N62,"Abro, Muhammad Wali",0.0,1.0,0.42909,0.57091
9,N7Q,"Ahmed, Ayube",0.0,0.0,0.92296,0.07704


In [31]:
#Saad Yasin - Difficut prediction idx 24
df.query('employee_code == "N7Z"')

,employee_code,efficient service,mobile expert,high school,information system,cash register,termination_type,rehired_,referral_flag,job_hopper,competitor_experience,accounting,arts,audio technician,blue collar_concentration,business,communication,computer systems,criminology,dental,economics,education_concentration,engineering,english,finance,general,healthcare,hospitality,human resource,interactive arts and technology,kinesiology,law,marketing,no education,other_concentration,physics,psychology,science,sociology,statistics,administrative,assistant manager,blue collar_work,cashier,cook,customer service representative,driver,education_work,financial services,fitness/sports,manager,no work,other_work,sales associate,technicians,telemarketers,highest_degree,background_highest_degree,country_highest_degree,sales_exp,customer_serv_exp,leader_ship_exp,hp_class
73,N7Z,0,0,0,0,0,New hire,No,0.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,bachelor,business,england,26.0,0.0,0.0,1.0


In [32]:
#SAHIL JOSHI - High performer idx 183
df.query('employee_code == "MQ2"')

,employee_code,efficient service,mobile expert,high school,information system,cash register,termination_type,rehired_,referral_flag,job_hopper,competitor_experience,accounting,arts,audio technician,blue collar_concentration,business,communication,computer systems,criminology,dental,economics,education_concentration,engineering,english,finance,general,healthcare,hospitality,human resource,interactive arts and technology,kinesiology,law,marketing,no education,other_concentration,physics,psychology,science,sociology,statistics,administrative,assistant manager,blue collar_work,cashier,cook,customer service representative,driver,education_work,financial services,fitness/sports,manager,no work,other_work,sales associate,technicians,telemarketers,highest_degree,background_highest_degree,country_highest_degree,sales_exp,customer_serv_exp,leader_ship_exp,hp_class
40,MQ2,0,0,0,0,0,Resignation,No,1.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,bachelor,psychology,canada,20.0,3.0,41.0,1.0


In [33]:
#Kayla Gregoire - Low performer idx 173
df.query('employee_code == "N5B"')

,employee_code,efficient service,mobile expert,high school,information system,cash register,termination_type,rehired_,referral_flag,job_hopper,competitor_experience,accounting,arts,audio technician,blue collar_concentration,business,communication,computer systems,criminology,dental,economics,education_concentration,engineering,english,finance,general,healthcare,hospitality,human resource,interactive arts and technology,kinesiology,law,marketing,no education,other_concentration,physics,psychology,science,sociology,statistics,administrative,assistant manager,blue collar_work,cashier,cook,customer service representative,driver,education_work,financial services,fitness/sports,manager,no work,other_work,sales associate,technicians,telemarketers,highest_degree,background_highest_degree,country_highest_degree,sales_exp,customer_serv_exp,leader_ship_exp,hp_class
173,N5B,0,0,0,0,0,New hire,No,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,certificate,NaN,canada,0.0,13.0,0.0,0.0
